# Personal Financial Assistant

## Objective

This notebook demonstrates the following:

- This notebook integrates an Assistant employing two tools: Function Calling and Code Interpreter. Leveraging a CSV file containing the user's investment portfolio, the system fetches real-time stock prices via ticker symbols using Function Calling. It conducts calculations through Code Interpreter and subsequently sends a report to the user via email using Function Calling

This tutorial uses the following Azure AI services:
- Access to Azure OpenAI Service - you can apply for access [here](https://aka.ms/oai/access)
- Azure OpenAI service - you can create it from instructions [here](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/create-resource)
- Azure OpenAI Studio - go to [https://oai.azure.com/](https://oai.azure.com/) to work with the Assistants API Playground
- A connection to the Azure OpenAI Service with a [Key and Endpoint](https://learn.microsoft.com/en-us/azure/ai-services/openai/chatgpt-quickstart)

Reference:
- Learn more about how to use Assistants with our [How-to guide on Assistants](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/assistant)
- [Assistants OpenAI Overview](https://platform.openai.com/docs/assistants/overview)

## Time

You should expect to spend 10-15 minutes running this sample. 

## About this example

This sample demonstrates the creation of an Azure OpenAI Assistant named "Portfolio Management Assistant" utilizing the Azure OpenAI API. The assistant is designed to act as a personal financial assistant, providing information and insights related to a user's investment portfolio. The script initiates a conversation with the assistant, guiding it through various financial queries and scenarios to showcase its capabilities.

### Data
This sample uses files from the folder [`data/`](./data/) in this repo. You can clone this repo or copy this folder to make sure you have access to these files when running the sample.

## Before you begin

### Installation

Install the following packages required to execute this notebook.

In [1]:
# Install the packages
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Parameters

In [2]:
import os

from dotenv import load_dotenv

load_dotenv("../.env")  # make sure to have the .env file in the root directory of the project

api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")
email_URI = os.getenv("EMAIL_URI")

should_cleanup: bool = False

## Run this Example

### Load the required libraries

In [3]:
import html
import io
import time
from datetime import datetime
from pathlib import Path
from typing import Iterable

import requests
import yfinance as yf
from openai import AzureOpenAI
from openai.types import FileObject
from openai.types.beta import Thread
from openai.types.beta.threads import Run
from openai.types.beta.threads.text_content_block import TextContentBlock
from openai.types.beta.threads.image_file_content_block import ImageFileContentBlock
from openai.types.beta.threads.messages import MessageFile
from PIL import Image

### Parameters

In [10]:
import os

from dotenv import load_dotenv

load_dotenv("../.env")  # make sure to have the .env file in the root directory of the project

api_endpoint = os.getenv("OPENAI_URI")
api_key = os.getenv("OPENAI_KEY")
api_version = os.getenv("OPENAI_VERSION")
api_deployment_name = os.getenv("OPENAI_GPT_DEPLOYMENT")

should_cleanup: bool = False

### Create an Azure OpenAI client

In [11]:
client = AzureOpenAI(api_key=api_key, api_version=api_version, azure_endpoint=api_endpoint)

### Get the latest stock price by ticker symbol using Yahoo Finance

In [6]:
def get_stock_price(symbol: str) -> float:
    stock = yf.Ticker(symbol)
    return stock.history(period="1d")["Close"].iloc[-1]

### Send an email using Logic Apps

In [ ]:
def send_logic_apps_email(to: str, content: str) -> None:
    # In this demo email was implemented using a Logic App with an HTTP trigger and M365 email action
    # Here are instructions on how to create an email endpoint using Logic Apps
    #   https://learn.microsoft.com/en-us/azure/app-service/tutorial-send-email?tabs=python
    try:
        json_payload = {"to": to, "content": html.unescape(content)}
        headers = {"Content-Type": "application/json"}
        response = requests.post(email_URI, json=json_payload, headers=headers)
        if response.status_code == 202:
            print("Email sent to: " + json_payload["to"])
    except:
        print("Failed to send email via Logic Apps")

### Define the Assistant tools

In [12]:
tools_list = [
    {"type": "code_interpreter"},
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": "Retrieve the latest closing price of a stock using its ticker symbol.",
            "parameters": {
                "type": "object",
                "properties": {"symbol": {"type": "string", "description": "The ticker symbol of the stock"}},
                "required": ["symbol"],
            },
        },
    },
    # {
    #     "type": "function",
    #     "function": {
    #         "name": "send_email",
    #         "description": "Sends an email to a recipient(s).",
    #         "parameters": {
    #             "type": "object",
    #             "properties": {
    #                 "to": {"type": "string", "description": "The email(s) the email should be sent to."},
    #                 "content": {"type": "string", "description": "The content of the email."},
    #             },
    #             "required": ["to", "content"],
    #         },
    #     },
    # },
]

### Upload the file(s)

In [8]:
DATA_FOLDER = "data/"

def upload_file(client: AzureOpenAI, path: str) -> FileObject:
    with Path(path).open("rb") as f:
        return client.files.create(file=f, purpose="assistants")

arr = os.listdir(DATA_FOLDER)
assistant_files = []
for file in arr:
    filePath = DATA_FOLDER + file
    assistant_files.append(upload_file(client, filePath))

file_ids = [file.id for file in assistant_files]

### Create an Assistant and a Thread

In [15]:
api_deployment_name = "gpt-4-turbo"



In [24]:
all_models = list(client.models.list())

for model in all_models:
    print(model.id)
    print(model.)



dall-e-3-3.0
dall-e-2-2.0
tts-001
tts-hd-001
whisper-001
gpt-35-turbo-0301
gpt-35-turbo-0613
gpt-35-turbo-1106
gpt-35-turbo-0125
gpt-35-turbo-instruct-0914
gpt-35-turbo-16k-0613
gpt-4-0125-Preview
gpt-4-1106-Preview
gpt-4-0314
gpt-4-0613
gpt-4-32k-0314
gpt-4-32k-0613
gpt-4-vision-preview
gpt-4-turbo-2024-04-09
gpt-4-turbo-jp
gpt-4o-2024-05-13
ada
text-similarity-ada-001
text-search-ada-doc-001
text-search-ada-query-001
code-search-ada-code-001
code-search-ada-text-001
text-embedding-ada-002
text-embedding-ada-002-2
babbage
babbage-002
text-similarity-babbage-001
text-search-babbage-doc-001
text-search-babbage-query-001
code-search-babbage-code-001
code-search-babbage-text-001
curie
text-similarity-curie-001
text-search-curie-doc-001
text-search-curie-query-001
davinci
davinci-002
text-davinci-003
text-similarity-davinci-001
text-search-davinci-doc-001
text-search-davinci-query-001
text-embedding-3-small
text-embedding-3-large
dall-e-3
dall-e-2
tts
tts-hd
whisper
gpt-35-turbo
gpt-35-tur

In [25]:

all_deployments = list(client.


AttributeError: 'AzureOpenAI' object has no attribute 'deployments'

In [20]:
all_files = list(client.files.list())

for file in all_files:
    print(file.id, file.filename, file.purpose)

assistant-3ywGBInm1sL3No8NeRLAUi3n portfolio.csv assistants
assistant-BVkNSQFYZ5uEWBydY0pIM4NU product_info_2.md assistants
assistant-0z6jSAjxbgqzbG1RNH7JCx1Z product_info_1.md assistants


In [16]:
assistant = client.beta.assistants.create(
    name="Portfolio Management Assistant",
    instructions="You are a personal securities trading assistant. Please be polite, professional, helpful, and friendly. "
    + "Use the provided portfolio CSV file to answer the questions. If question is not related to the portfolio or you cannot answer the question, say, 'contact a representative for more assistance.'"
    + "If the user asks for help or says 'help', provide a list of sample questions that you can answer.",
    tools=tools_list,
    model=api_deployment_name,
    file_ids=file_ids,
)

thread = client.beta.threads.create()

BadRequestError: Error code: 400 - {'error': {'message': "Unknown parameter: 'file_ids'.", 'type': 'invalid_request_error', 'param': 'file_ids', 'code': 'unknown_parameter'}}

### Process Function calling

In [ ]:
def call_functions(client: AzureOpenAI, thread: Thread, run: Run) -> None:
    print("Function Calling")
    required_actions = run.required_action.submit_tool_outputs.model_dump()
    print(required_actions)
    tool_outputs = []
    import json

    for action in required_actions["tool_calls"]:
        func_name = action["function"]["name"]
        arguments = json.loads(action["function"]["arguments"])

        if func_name == "get_stock_price":
            output = get_stock_price(symbol=arguments["symbol"])
            tool_outputs.append({"tool_call_id": action["id"], "output": output})
        elif func_name == "send_email":
            print("Sending email...")
            email_to = arguments["to"]
            email_content = arguments["content"]
            send_logic_apps_email(email_to, email_content)

            tool_outputs.append({"tool_call_id": action["id"], "output": "Email sent"})
        else:
            raise ValueError(f"Unknown function: {func_name}")

    print("Submitting outputs back to the Assistant...")
    client.beta.threads.runs.submit_tool_outputs(thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

### Format and display the Assistant Messages for text and images

In [ ]:
def read_assistant_file(file_id:str):
    response_content = client.files.content(file_id)
    return response_content.read()

def print_messages(messages: Iterable[MessageFile]) -> None:
    message_list = []

    # Get all the messages till the last user message
    for message in messages:
        message_list.append(message)
        if message.role == "user":
            break

    # Reverse the messages to show the last user message first
    message_list.reverse()

    # Print the user or Assistant messages or images
    for message in message_list:
        for item in message.content:
            # Determine the content type
            if isinstance(item, TextContentBlock):
                print(f"{message.role}:\n{item.text.value}\n")
                file_annotations = item.text.annotations
                if file_annotations:
                    for annotation in file_annotations:
                        file_id = annotation.file_path.file_id
                        content = read_assistant_file(file_id)
                        print(f"Annotation Content:\n{str(content)}\n")
            elif isinstance(item, ImageFileContentBlock):
                # Retrieve image from file id                
                data_in_bytes = read_assistant_file(item.image_file.file_id)
                # Convert bytes to image
                readable_buffer = io.BytesIO(data_in_bytes)
                image = Image.open(readable_buffer)
                # Resize image to fit in terminal
                width, height = image.size
                image = image.resize((width // 2, height // 2), Image.LANCZOS)
                # Display image
                image.show()

### Process the user messages

In [ ]:
def process_prompt(prompt: str) -> None:
    client.beta.threads.messages.create(thread_id=thread.id, role="user", content=prompt)
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please address the user as Jane Doe. The user has a premium account. Be assertive, accurate, and polite. Ask if the user has further questions. "
        + "The current date and time is: "
        + datetime.now().strftime("%x %X")
        + ". ",
    )
    print("processing ...")
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status == "completed":
            # Handle completed
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            print_messages(messages)
            break
        if run.status == "failed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            answer = messages.data[0].content[0].text.value
            print(f"Failed User:\n{prompt}\nAssistant:\n{answer}\n")
            # Handle failed
            break
        if run.status == "expired":
            # Handle expired
            print(run)
            break
        if run.status == "cancelled":
            # Handle cancelled
            print(run)
            break
        if run.status == "requires_action":
            call_functions(client, thread, run)
        else:
            time.sleep(5)

### Have a conversation with the Assistant

In [ ]:
process_prompt("Based on the provided portfolio, what investments do I own?")

In [ ]:
process_prompt("What is the value of my portfolio?")

In [ ]:
process_prompt("What is my best and worst investment?")

In [ ]:
process_prompt("Chart the realized gain or loss of my investments.")

In [ ]:
process_prompt(
    "Please send a report to name@contoso.com with the details for each stock based on the latest stock prices, and list the best and worst performing stocks in my portfolio."
)

### Cleaning up

In [ ]:
if should_cleanup:
    client.beta.assistants.delete(assistant.id)
    client.beta.threads.delete(thread.id)
    for file in assistant_files:
        client.files.delete(file.id)